# Análise do GRASP MCMC

$$G =10 \log_{10}\left[\left\Vert 2\pi\int_{-\theta_0}^{\theta_0} E_{CO} \cos\theta\;d\theta \right\Vert^2\right]$$    

$$G_{max} = 10 \log_{10} \frac{\Vert E\Vert_{max}}{\int_{-\theta_0}^{\theta_0} \Vert E \Vert^2 \cos\theta\;d\theta}$$

## Inicialização

In [1]:
import os
import sys
import glob
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display
#-----
import time
import subprocess
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.path.split(os.getcwd())[0], "src")))
import grasp
#-----
%matplotlib inline
plt.rcParams["figure.figsize"] = (16, 6)
InteractiveShell.ast_node_interactivity = "all"

## Emcee Analysis

In [21]:
def make_tor_mcmc(*thetas):
    dirname = "../grasp/STANDARD/tmp_mcmc/"
    # cria pasta
    if not os.path.isdir(dirname):
        os.mkdir(dirname)
    filename = dirname + str(np.random.randint(10000000)) + ".tor"
    feed_string1 = grasp.move_feed(y=thetas[0])
    # grava arquivo tor
    grasp._make_tor(filename, feed_string1, "../grasp/STANDARD/BINGO_CUT_fast.tor", 15, 7)
    sec_string = grasp.translate_secondary(y=thetas[1])
    grasp._make_tor(filename, sec_string, filename, 7, 0)
    return filename

In [ ]:
params = np.round(np.linspace(-.5, .5, 7), 1)
files = []
fys = []
sys = []
gains = []
for ii, fy in enumerate(params):
    for jj, sy in enumerate(params):
        tor_file = make_tor_mcmc(fy, sy)
        _ = grasp.run_grasp(tor_file, daemon=False)
        cut_file = "../" + tor_file.split(".")[-2] + ".cut"
        df = grasp.read_cut(cut_file)
        gain = grasp.gain_dB(df)
        files.append(cut_file)
        fys.append(fy)
        sys.append(sy)
        gains.append(gain)
result = pd.DataFrame({"cut_files": files, "fy": fys}, "sy": sys, "Gain": gains)

In [ ]:
best_fit = grasp.read_cut("../grasp/STANDARD/job_feed_translation_x_0/dual_cut01")
best_params = [0, 0]

def residual(df, n_params):
    result = np.sqrt(np.sum( ((df.Eco**2 + df.ImEco**2) - (best_fit.Eco**2 + best_fit.ImEco**2)) ** 2)) / (best_fit.shape[0] - n_params)
    return result
    
    
def log_prior(*thetas, t1_range=[-5, 5], t2_range=[-5, 5]):
    if thetas[0] > t1_range[0] and thetas[0] < t1_range[1]:
        if thetas[1] > t2_range[0] and thetas[1] < t2_range[1]:
            result = 0
        else:
            result = -np.inf
    else:
        result = -np.inf
    return result


def log_likelyhood(*thetas):
    tor_file = make_tor_mcmc(*thetas)
    grasp.run_grasp(tor_file, daemon=False)
    df = grasp.read_cut("../grasp/STANDARD/tmp_mcmc/"+tor_file + ".cut")
    result = - residual(df, len(thetas))
    return result


def log_posterior(thetas):
    result = log_prior(thetas) + log_likelyhood(thetas)
    return result

In [ ]:
tor_file = make_tor_mcmc(1, 1)
tor_file
#grasp.run_grasp(tor_file, daemon=False)

In [ ]:
df = grasp.read_cut("../grasp/STANDARD/tmp_mcmc/dual_cut01")

In [ ]:
ndim = len(best_params)
nburn = 1
nwalkers = 2
nsteps = 20
starting_guesses = np.random.normal(loc=0, scale=0.1, size=(nwalkers, ndim) )


In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=[xx, yy], pool=pool)
sampler.run_mcmc(starting_guesses, nsteps, progress=True);

In [ ]:
samples = sampler.get_chain(discard=nburn, flat=True)
thetas = np.zeros(ndim)
errs = np.zeros(ndim)
for ii in np.arange(ndim):
    mcmc = np.percentile(samples[:, ii], [5, 50, 95])
    thetas[ii] = mcmc[1]
    errs[ii] = (mcmc[2] - mcmc[0])/2